In [109]:
import mido

mid = mido.MidiFile("StorkeTower.mid", clip=True)


def roundPartial(value, resolution):
    return round(value / resolution) * resolution


patterns = []
prev_tempo = None
time = 0
for msg in mid:
    time += msg.time
    if msg.type == "program_change":
        print(msg)
    if msg.type == "set_tempo":
        print(msg)
        print("Tempo set to", msg.tempo)
        bpm = 6e7 / msg.tempo
        if prev_tempo != msg.tempo:
            if prev_tempo is not None:
                print("TIME", msg.time)
                length = time * patterns[-1]["bpm"] / 60 / 4
                print("LENGTH", length)
                patterns[-1]["time"] = time
                patterns[-1]["cumulative_time"] = time if len(patterns) <= 1 else time + patterns[-2]["cumulative_time"]
                patterns[-1]["length"] = roundPartial(length, 0.25)
                time = 0 # time resets every time tempo changes
            patterns.append({"bpm": bpm})
        print("bpm:", round(bpm, 1))
        prev_tempo = msg.tempo
else:
    length = time * patterns[-1]["bpm"] / 60 / 4
    print("LENGTH", length)
    patterns[-1]["time"] = time
    patterns[-1]["cumulative_time"] = time if len(patterns) <= 1 else time + patterns[-2]["cumulative_time"]
    patterns[-1]["length"] = roundPartial(length, 0.25)
    print("TIME", time)
    print("length", length)
print(patterns)

MetaMessage('set_tempo', tempo=1224489, time=0)
Tempo set to 1224489
bpm: 49.0
program_change channel=0 program=14 time=0
MetaMessage('set_tempo', tempo=1000000, time=0.00255101875)
Tempo set to 1000000
TIME 0.00255101875
LENGTH 5.750000000000001
bpm: 60.0
MetaMessage('set_tempo', tempo=1000000, time=0.0020833333333333333)
Tempo set to 1000000
bpm: 60.0
LENGTH 7.500000000000001
TIME 30.000000000000004
length 7.500000000000001
[{'bpm': 49.000031850020704, 'time': 28.163247000000005, 'cumulative_time': 28.163247000000005, 'length': 5.75}, {'bpm': 60.0, 'time': 30.000000000000004, 'cumulative_time': 58.16324700000001, 'length': 7.5}]


In [110]:
# Initialize lists to store MIDI data and output
midi_events = []
outputs = [[]]
i = 0

# Extract relevant MIDI events and convert them to dictionaries
for event in mid:
    midi_events.append(event.dict())

# Convert delta times to absolute times
current_time = 0
for event in midi_events:
    event["time"] += current_time
    current_time = event["time"]

    # Convert note_on events with 0 velocity to note_off events
    if event["type"] == "note_on" and event["velocity"] == 0:
        event["type"] = "note_off"

    # Prepare the event data for output
    event_data = []
    if event["type"] in ["note_on", "note_off"]:
        event_data = [event["type"], event["note"], event["time"], event["channel"]]
        if event["time"] < patterns[i]["cumulative_time"]:
            if i >= 1:
                event_data[2] -= patterns[i - 1]["cumulative_time"]
            outputs[i].append(event_data)
        else:
            i += 1
            event_data[2] -= patterns[i - 1]["cumulative_time"]
            outputs.append([event_data])
        print("event_data", event_data)

# Display the processed MIDI events
for i, output in enumerate(outputs):
    print("Pattern", i)
    for event in output:
        print(event)

# Print the ticks per beat of the MIDI file
print(mid.ticks_per_beat)

event_data ['note_on', 60, 0, 0]
event_data ['note_off', 60, 1.22193798125, 0]
event_data ['note_on', 57, 1.224489, 0]
event_data ['note_off', 57, 2.44642698125, 0]
event_data ['note_on', 48, 2.4489780000000003, 0]
event_data ['note_off', 48, 4.8954049812500005, 0]
event_data ['note_on', 48, 4.897956000000001, 0]
event_data ['note_off', 48, 6.119893981250001, 0]
event_data ['note_on', 53, 6.122445000000001, 0]
event_data ['note_off', 53, 7.344382981250001, 0]
event_data ['note_on', 57, 7.346934000000001, 0]
event_data ['note_off', 57, 8.568871981250002, 0]
event_data ['note_on', 50, 8.571423000000001, 0]
event_data ['note_off', 50, 9.79336098125, 0]
event_data ['note_on', 57, 9.795912, 0]
event_data ['note_off', 57, 12.24233898125, 0]
event_data ['note_on', 59, 12.24489, 0]
event_data ['note_off', 59, 13.466827981249999, 0]
event_data ['note_on', 57, 13.469378999999998, 0]
event_data ['note_off', 57, 15.91580598125, 0]
event_data ['note_on', 60, 15.918356999999999, 0]
event_data ['note

In [111]:
def generate_tracks(output):
    # create a 31-element list of empty lists called "tracks"
    tracks = [[] for _ in range(31)]

    # put all notes in the right track
    for event in output:
        tracks[event[3]].append(event)

    # put all notes in the right order
    for track in tracks:
        track.sort(key=lambda event: event[2])

    # print all tracks
    for track in tracks:
        print(track)
    print()

    return tracks


tracks_list = []
for output in outputs:
    tracks_list.append(generate_tracks(output))

[['note_on', 60, 0, 0], ['note_off', 60, 1.22193798125, 0], ['note_on', 57, 1.224489, 0], ['note_off', 57, 2.44642698125, 0], ['note_on', 48, 2.4489780000000003, 0], ['note_off', 48, 4.8954049812500005, 0], ['note_on', 48, 4.897956000000001, 0], ['note_off', 48, 6.119893981250001, 0], ['note_on', 53, 6.122445000000001, 0], ['note_off', 53, 7.344382981250001, 0], ['note_on', 57, 7.346934000000001, 0], ['note_off', 57, 8.568871981250002, 0], ['note_on', 50, 8.571423000000001, 0], ['note_off', 50, 9.79336098125, 0], ['note_on', 57, 9.795912, 0], ['note_off', 57, 12.24233898125, 0], ['note_on', 59, 12.24489, 0], ['note_off', 59, 13.466827981249999, 0], ['note_on', 57, 13.469378999999998, 0], ['note_off', 57, 15.91580598125, 0], ['note_on', 60, 15.918356999999999, 0], ['note_off', 60, 17.140294981249998, 0], ['note_on', 52, 17.142846, 0], ['note_off', 52, 18.36478398125, 0], ['note_on', 55, 18.367335, 0], ['note_off', 55, 19.58927298125, 0], ['note_on', 53, 19.591824000000003, 0], ['note_of

In [113]:
import uuid


def format_tracks(tracks):
    tracks_with_duration = [[] for _ in range(31)]

    # calculate the duration of each note
    for event in range(len(tracks)):
        for j in range(len(tracks[event])):
            if tracks[event][j][0] == "note_on":
                for k in range(j, len(tracks[event])):
                    if (
                        tracks[event][k][0] == "note_off"
                        and tracks[event][k][1] == tracks[event][j][1]
                    ):
                        tracks_with_duration[event].append(
                            {
                                "id": str(uuid.uuid4()),
                                "pitch": float(tracks[event][j][1]),
                                "time": tracks[event][j][2],
                                "duration": round(
                                    (tracks[event][k][2] - tracks[event][j][2])
                                    * bpm
                                    / 60,
                                    4,
                                ),
                            }
                        )
                        break

    # print all tracks with duration
    for event in tracks_with_duration:
        print(event)
    print()

    return tracks_with_duration


tracks_with_duration_list = []
for tracks in tracks_list:
    tracks_with_duration_list.append(format_tracks(tracks))

[{'id': '19c51548-683b-4cfe-9f09-7c27a10f4674', 'pitch': 60.0, 'time': 0, 'duration': 1.2219}, {'id': '1e921169-83a9-44a2-a740-9ba642dac956', 'pitch': 57.0, 'time': 1.224489, 'duration': 1.2219}, {'id': '023a6bb5-4a16-416b-978f-6f7cd7dd6374', 'pitch': 48.0, 'time': 2.4489780000000003, 'duration': 2.4464}, {'id': '85b78fe3-9c8b-4695-8633-c7e735288de4', 'pitch': 48.0, 'time': 4.897956000000001, 'duration': 1.2219}, {'id': '0a36063e-9aa1-4720-be7c-1a0a960ecbaa', 'pitch': 53.0, 'time': 6.122445000000001, 'duration': 1.2219}, {'id': 'bfa4aef8-9795-4d2c-9fbb-a371a784f341', 'pitch': 57.0, 'time': 7.346934000000001, 'duration': 1.2219}, {'id': '24f7a124-8a2a-48ac-a2b5-9f92aafb7e46', 'pitch': 50.0, 'time': 8.571423000000001, 'duration': 1.2219}, {'id': '752e1072-66c7-4861-a647-8437f800e380', 'pitch': 57.0, 'time': 9.795912, 'duration': 2.4464}, {'id': 'fe792bc1-94a9-4e89-9dca-c4203d3ac15c', 'pitch': 59.0, 'time': 12.24489, 'duration': 1.2219}, {'id': 'f7c09823-20ab-4a13-9ebc-a3786a6d288e', 'pit

In [84]:
total_time = 0
for msg in mid:
    total_time += msg.time

print("Total time:", total_time)

size = round(
    total_time * bpm / 60 / 4
)  # because 60 seconds in a minute (we use bpm <- minute) and 4 beats in a bar

print("Size:", size)

Total time: 58.163247000000005
Size: 15


In [114]:
# program_change channel=0 program=36 time=0
# program_change channel=1 program=0 time=0
# program_change channel=2 program=32 time=0
# program_change channel=3 program=0 time=0
# program_change channel=4 program=81 time=0
# program_change channel=5 program=80 time=0
# program_change channel=6 program=81 time=0
# program_change channel=7 program=42 time=0
# program_change channel=8 program=64 time=0
# program_change channel=9 program=0 time=0
# program_change channel=10 program=64 time=0
# program_change channel=11 program=64 time=0
# program_change channel=12 program=64 time=0
# program_change channel=13 program=64 time=0
# program_change channel=14 program=64 time=0
# program_change channel=15 program=64 time=0
channel_patches = [
    35,
    31,
    30,
    31,
    9,
    80,
    5,
    16,
    11,
    33,
    11,
    11,
    11,
    11,
    11,
    11,
]

In [53]:
res = {
    "value0": {
        "patterns": [
            {
                "bpm": patterns[i]["bpm"],
                "length": patterns[i]["length"],
                "name": "",
                "id": str(uuid.uuid4()),
                "tracks": tracks_with_duration_list[i],
            }
            for i in range(len(patterns))
        ],
        "channels": [
            {
                "patch": patch,
                "atk": 0.0010000000474974514,
                "rel": 0.12999999523162843,
                "volume": 1.0,
                "pan": 0.0,
                "lp": 20000.0,
                "tune": 0.0,
                "soft": True
            }
            for patch in channel_patches
        ]
    }
}

In [54]:
# output to song.json
import json

with open("song.json", "w") as f:
    json.dump(res, f, indent=4)
    